In [32]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/DS_I"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DS_I


In [2]:
import requests
import json
import time
import pandas as pd
import re
import os
import random
import urllib3
import multiprocessing as mp


In [3]:
from datetime import datetime
from bs4 import BeautifulSoup
from tqdm import tqdm

urllib3.disable_warnings()
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

In [4]:
def GetRes(url):
    res =  requests.get(url, headers=headers)
    res.encoding='utf-8'
    return res

In [5]:
def GetANews(link):
    try :
        res = GetRes(link)
        soup = BeautifulSoup(res.text)
        title = soup.find('h1', attrs={'id':'node-title'}).text
        content = '\n'.join(i.text for i in soup.find('div',class_='field--name-field-pr-body').find_all('p'))
        date = soup.find('span', class_="date-display-single")['content']
    except:
        return None
    
    data = {'TITLE':title,'CONTENT':content,'DATE': date}
    
    return data

In [39]:
def GetLinks(link):
    res = GetRes(link)
    soup = BeautifulSoup(res.text)
    # print(soup)
    news_soup_list = soup.find_all('div',class_='views-row')
    news_list = []
    for i in news_soup_list:
        url = i.find('a')['href']
        if url and 'pr' in url: 
            news_list.append(url)
    return news_list

In [46]:
def CrawlNews(start_page, end_page):
    Links = []
    k = 0
    print(start_page, end_page)

    for i in range(start_page,end_page):
        # print(i)
        #k = unisleep(k)
        url = 'https://www.justice.gov/news?page='+str(i)
        tmp = GetLinks(url)
        Links.extend(tmp)
    # print(Links)
    
    responses =[]
    res_list=[]
    news_list =[]
    for link in tqdm(Links):
        link = 'https://www.justice.gov' + link
        # response = GetRes(link)
        sub_list = GetANews(link)
        if sub_list is not None:
            news_list.append(sub_list)

    return  news_list

In [57]:
page_num = 0
pages_one_time = 10
News = pd.DataFrame() 
final_page = 480
dic_list=[]

In [58]:
while page_num < final_page:
    if page_num+pages_one_time > final_page:
        dic_list.extend(CrawlNews(page_num, final_page))
    else:
        dic_list.extend(CrawlNews(page_num, page_num+pages_one_time))
    page_num+=pages_one_time

News = pd.DataFrame.from_dict(dic_list)

400 410


100%|██████████| 217/217 [01:03<00:00,  3.39it/s]


410 420


100%|██████████| 227/227 [00:22<00:00, 10.03it/s]


420 430


...

100%|██████████| 227/227 [00:38<00:00,  5.84it/s]


460 470


100%|██████████| 222/222 [00:28<00:00,  7.82it/s]


470 480


100%|██████████| 223/223 [00:27<00:00,  8.18it/s]


In [60]:
fout_name = "/content/drive/MyDrive/DS_I/data/final.json"
f = open(fout_name,'w')
json_ver = News.to_json(force_ascii=False)

f.write(json_ver)

7255397